In [ ]:
pip install spacy==2.3.5

In [ ]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

In [ ]:
pip install gensim

### Dataframe

In [79]:
import pandas as pd
from datetime import datetime
df = pd.DataFrame(columns = ["phone_num", "text", "keywords", "time", "predicted_event_ID", "action"])
df.loc[len(df)] = ([99998888, "hello, There's a fire at Bishan", ["fire", "Bishan"], datetime.now(), 1, "Emergency"])
df

,phone_num,text,keywords,time,predicted_event_ID,action
0,99998888,"hello, There's a fire at Bishan","[fire, Bishan]",2021-06-09 19:24:25.229798,1,Emergency


In [48]:
'''
from collections import Counter
import math

counterA = Counter(["Bishan", "fire", "Bishan"])
counterB = Counter(["Bishan", "emmergency", "fire"])


def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

print(counter_cosine_similarity(counterA, counterB))
'''

0.7745966692414834


### Similarity check

In [53]:
'''Similarity between two sentences'''
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhouentong/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0.9999999999999998
0.7092972666062738
0.0


/Users/zhouentong/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in st

### Actual function

In [78]:
from datetime import datetime
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

#text = open('audio.txt', 'r').read()

event_id = 10000
def auto_operator(text, phone_num):
    time = datetime.now()
    
    ## identify keywords
    doc = nlp(text)
    keywords = []
    for token in doc:
        if token.pos_=='NOUN' or token.pos_ == 'PROPN':
            keywords.append(token)
    action = "Emergency"  
    ## similarity check (can change)
    global event_id
    current_event_id = event_id
    for index, row in df.iterrows():
        similarity = cosine_sim(row["text"], text)
        # change threshold
        if similarity > 0.8:
            current_event_id = row["predicted_event_ID"]
            action = "auto reply: xxx"
            break
    event_id += 1 if current_event_id == event_id else 0
    
    ## decide on action - fire, rescue, medical, terrorism
    
    ## repeated calls - auto reply
    
    ## Store in df
    df.loc[len(df)] = ([phone_num, text, keywords, datetime.now(), current_event_id, action])
    return action

### Test

In [80]:
auto_operator("I had a fever", 99999999)
#df = df.drop(1)
df

,phone_num,text,keywords,time,predicted_event_ID,action
0,99998888,"hello, There's a fire at Bishan","[fire, Bishan]",2021-06-09 19:24:25.229798,1,Emergency
1,99999999,I had a fever,[fever],2021-06-09 19:24:30.068486,10000,Emergency


In [81]:
auto_operator("I was at Utown and I saw an old man fall down", 90099999)
df

,phone_num,text,keywords,time,predicted_event_ID,action
0,99998888,"hello, There's a fire at Bishan","[fire, Bishan]",2021-06-09 19:24:25.229798,1,Emergency
1,99999999,I had a fever,[fever],2021-06-09 19:24:30.068486,10000,Emergency
2,90099999,I was at Utown and I saw an old man fall down,"[Utown, man]",2021-06-09 19:24:31.930524,10001,Emergency


In [82]:
auto_operator("A old man fell down at uTown, please help!", 90099999)
df

,phone_num,text,keywords,time,predicted_event_ID,action
0,99998888,"hello, There's a fire at Bishan","[fire, Bishan]",2021-06-09 19:24:25.229798,1,Emergency
1,99999999,I had a fever,[fever],2021-06-09 19:24:30.068486,10000,Emergency
2,90099999,I was at Utown and I saw an old man fall down,"[Utown, man]",2021-06-09 19:24:31.930524,10001,Emergency
3,90099999,"A old man fell down at uTown, please help!","[man, uTown]",2021-06-09 19:24:33.596706,10002,Emergency


In [84]:
auto_operator("I saw a old man fell down at uTown, please help!", 90099999)
df

,phone_num,text,keywords,time,predicted_event_ID,action
0,99998888,"hello, There's a fire at Bishan","[fire, Bishan]",2021-06-09 19:24:25.229798,1,Emergency
1,99999999,I had a fever,[fever],2021-06-09 19:24:30.068486,10000,Emergency
2,90099999,I was at Utown and I saw an old man fall down,"[Utown, man]",2021-06-09 19:24:31.930524,10001,Emergency
3,90099999,"A old man fell down at uTown, please help!","[man, uTown]",2021-06-09 19:24:33.596706,10002,Emergency
4,90099999,"I saw a old man fell down at uTown, please sen...","[man, uTown]",2021-06-09 19:25:01.989697,10003,Emergency
5,90099999,"I saw a old man fell down at uTown, please help!","[man, uTown]",2021-06-09 19:29:09.784814,10002,auto reply: xxx


In [70]:
cosine_sim("A old man fell down at uTown help", "I saw an old man fall down")

0.29121941856368966

In [76]:
text = "A old man fell down at uTown, please help!"
doc = nlp(text)
keywords = []
for token in doc:
    if token.pos_=='NOUN' or token.pos_ == 'PROPN':
        keywords.append(token)
print(keywords)
for token in doc:
    print(token.text,'->',token.pos_)

[man, uTown]
A -> DET
old -> ADJ
man -> NOUN
fell -> VERB
down -> ADP
at -> ADP
uTown -> PROPN
, -> PUNCT
please -> INTJ
help -> VERB
! -> PUNCT
